In [1]:
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, insert
import os


In [2]:
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver="ODBC Driver 18 for SQL Server"
connection_string = (f"mssql+pyodbc://{azure_client_id}:{azure_client_secret}@{server}:1433/{database}"
f"?driver={driver}"
"&authentication=ActiveDirectoryServicePrincipal"
"&timeout=120"
"&Encrypt=yes"
"&TrustServerCertificate=no"
)

In [ ]:
engine2 = create_engine(connection_string, connect_args={'timeout': 120})


In [41]:
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import FileSystemClient
import pyodbc, struct
import os
from msal import ConfidentialClientApplication
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver="ODBC Driver 18 for SQL Server"
AUTHORITY = f"https://login.microsoftonline.com/{azure_tenant_id}"
connection_string = f'Driver={driver};Server=tcp:{server},1433;Database={database};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'
credential = ClientSecretCredential(tenant_id=azure_tenant_id,
                              client_id=azure_client_id,
                              client_secret=azure_client_secret)

token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h
conn = pyodbc.connect(connection_string, attrs_before={1256: token_struct})
cursor = conn.cursor()
cursor.execute("SELECT * FROM [dbo].[new_table] WHERE name = 'Test User'")

row = cursor.fetchall()
for ro in row:
    print(f"{ro.age}, {ro.name}")
# cursor.execute("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')")
# conn.commit()

25, Test User


In [49]:
from sqlmodel import SQLModel, Session, Field, create_engine, select
from azure.identity import ClientSecretCredential
import os
import struct

# --- Azure credentials ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")     # full Fabric/SQL hostname, e.g. "xxx.sql.azuredatabases.net"
database = os.getenv("DATABASE")

# --- Get access token ---
credential = ClientSecretCredential(
    tenant_id=azure_tenant_id,
    client_id=azure_client_id,
    client_secret=azure_client_secret
)
token = credential.get_token("https://database.windows.net/.default").token
token_bytes = token.encode("utf-16-le")  # utf-16-le = required by SQL Server
token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)

# --- SQLAlchemy engine using ODBC Driver ---
odbc_str = (
    f"mssql+pyodbc://@{server}:1433/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&Encrypt=yes&TrustServerCertificate=no&Connection+Timeout=30"
)

from sqlalchemy import event

engine = create_engine(odbc_str)

# Inject access token on connect
@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    cparams["attrs_before"] = {1256: token_struct}  # 1256 = SQL_COPT_SS_ACCESS_TOKEN

# --- Define SQLModel (example) ---
class User(SQLModel, table=True):
    __tablename__ = "new_table"
    __table_args__ = {"schema": "dbo","extend_existing": True}
    age: int | None = Field(default=None, primary_key=True)
    name: str | None = None

# --- Query with SQLModel ---
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    stmt = select(User).where(User.name == "Test User")
    results = session.exec(stmt).all()
    for row in results:
        print(row)


DBAPIError: (pyodbc.Error) ('FA005', '[FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0) (SQLDriverConnect); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [78]:
from sqlmodel import SQLModel, Session, select
from azure.identity import ClientSecretCredential
from sqlalchemy import create_engine, event
import os, struct

# --- Azure creds ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")     # e.g. "xxx.datawarehouse.fabric.microsoft.com"
database = os.getenv("DATABASE")

# --- Get access token ---
credential = ClientSecretCredential(
    tenant_id=azure_tenant_id,
    client_id=azure_client_id,
    client_secret=azure_client_secret,
)
token = credential.get_token("https://database.windows.net/.default").token
token_bytes = token.encode("utf-16-le")
token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)

# --- Connection string (NO Authentication here!) ---
# odbc_str = (
#     f"mssql+pyodbc://@{server}:1433/{database}"
#     "?driver=ODBC+Driver+18+for+SQL+Server"
#     "&Encrypt=yes&TrustServerCertificate=no&Connection+Timeout=30"
# )
# odbc_str = (f"mssql+pyodbc://{azure_client_id}:{azure_client_secret}@{server}:1433/{database}"
# f"?driver={driver}"
# "&authentication=ActiveDirectoryServicePrincipal"
# "&timeout=120"
# "&Encrypt=yes"
# "&TrustServerCertificate=no"
# "&Authentication=ActiveDirectoryAccessToken"
# )
import urllib
driver_name = '{ODBC Driver 18 for SQL Server}'
server_name = os.getenv("SERVER")
database_name = os.getenv("DATABASE")
connection_string = 'Driver={};Server=tcp:{}.database.windows.net,1433;Database={};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'.format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
odbc_str = "mssql+pyodbc:///?odbc_connect={0}".format(params)

# --- Create engine ---
engine = create_engine(odbc_str, echo=True)
# engine = create_engine(odbc_str, echo=True, connect_args = {"attrs_before":{1256: token_struct}})
# --- Inject the access token ---
@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    cparams["attrs_before"] = {1256: token_struct}  # SQL_COPT_SS_ACCESS_TOKEN

# # --- Example table model ---
# from sqlmodel import Field

class New_table(SQLModel, table=True):
    __tablename__ = "new_table"
    __table_args__ = {"schema": "dbo", "extend_existing": True}

    age: int | None = Field(default=None, primary_key=True)
    name: str | None = None


new_user = New_table(age=30, name="You")

with Session(engine) as session:
    session.add(new_user)
    session.commit()
# --- Query ---
with Session(engine) as session:
    stmt = select(New_table)
    results = session.exec(stmt).all()
    for row in results:
        print(row)


OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Kjenner ingen slik vert.\r\n (11001) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to tcp:b5li4afpbdwezicwwnkeoubzse-m3nesqac3ueepkmopydvg35o3y.datawarehouse.fabric.microsoft.com.database.windows.net,1433. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (11001)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
import os
import struct
import urllib
from sqlmodel import SQLModel, Field, Session, select, Relationship
from sqlalchemy import create_engine, event, Column
from azure.identity import DefaultAzureCredential, ClientSecretCredential
import uuid
from sqlalchemy.dialects.mssql import UNIQUEIDENTIFIER
# --- Azure creds ---
driver_name = "{ODBC Driver 18 for SQL Server}"
server_name = os.getenv("SERVER")      # e.g. "xxx.datawarehouse.fabric.microsoft.com"
database_name = os.getenv("DATABASE")

# --- Connection string (NO auth here, token comes later) ---
connection_string = (
    "Driver={};Server=tcp:{},1433;Database={};Encrypt=yes;"
    "TrustServerCertificate=no;Connection Timeout=30"
).format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
odbc_str = f"mssql+pyodbc:///?odbc_connect={params}"

# --- Get SQLAlchemy engine ---
engine = create_engine(odbc_str, echo=True)

# --- Token injection hook ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
credential = ClientSecretCredential(tenant_id=azure_tenant_id,client_id=azure_client_id,client_secret=azure_client_secret)  # or ClientSecretCredential if you prefer

@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    print("🔑 Requesting new Entra ID token for SQL Server...")
    token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("utf-16-le")
    token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256
    cparams["attrs_before"] = {SQL_COPT_SS_ACCESS_TOKEN: token_struct}

# --- SQLModel table ---
class Client(SQLModel, table=True):
    __tablename__ = "new_table_id"
    __table_args__ = {"schema": "dbo"}

    id: uuid.UUID = Field(
        default_factory=uuid.uuid4,
        sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
    )
    name: str
    age: int

    # one-to-many relationship


class Orders(SQLModel, table=True):
    __tablename__ = "orders_test"
    __table_args__ = {"schema": "dbo"}

    id: uuid.UUID = Field(
        default_factory=uuid.uuid4,
        sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
    )
    amount: int
    product: str
    

    customer_id: uuid.UUID = Field(
        foreign_key="dbo.new_table_id.id",  # 👈 link to users
    )



with Session(engine) as session:
    # Join with select()
    stmt = (
        select(Client, Orders)
        .join(Orders, Client.id == Orders.customer_id)
        .where(Client.name == "mee")
    )

    results = session.exec(stmt).all()

    for user, order in results:
        print(f"{user.name} (age {user.age}) bought {order.product} for {order.amount}")
# print(new_table.schema())


# with Session(engine) as session:
#     new_user = new_table(age=5, name="Ung2")   # id will be auto-generated as UUID
#     session.add(new_user)
#     session.commit()
#     session.refresh(new_user)
#     print(new_user.age, new_user.name)


# # # --- Query example ---
# with Session(engine) as session:
#     stmt = select(new_table)
#     results = session.exec(stmt).all()
#     for row in results:
#         print(f"{row.id}, {row.age}, {row.name}")


🔑 Requesting new Entra ID token for SQL Server...
2025-09-23 10:35:09,352 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-09-23 10:35:09,352 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-23 10:35:09,359 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-09-23 10:35:09,360 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
2025-09-23 10:35:09,377 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-09-23 10:35:09,378 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ()
2025-09-23 10:35:09,385 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-09-23 10:35:09,385 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()
2025-09-23 10:35:09,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-23 10:35:09,401 INFO sqlalchemy.engine.Engine SELECT dbo.new_table_id.id, dbo.new_table_id.name, dbo.new_table_id.age

In [3]:
with Session(engine) as session:
    # Join with select()
    stmt = (
        select(Client, Orders)
        .join(Orders, Client.id == Orders.customer_id, isouter=True)
    )

    results = session.exec(stmt).all()

    for user, order in results:
        print(f"{user.name} (age {user.age}) bought {order.product} for {order.amount}")

2025-09-23 10:36:00,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-23 10:36:00,342 INFO sqlalchemy.engine.Engine SELECT dbo.new_table_id.id, dbo.new_table_id.name, dbo.new_table_id.age, dbo.orders_test.id AS id_1, dbo.orders_test.amount, dbo.orders_test.product, dbo.orders_test.customer_id 
FROM dbo.new_table_id LEFT OUTER JOIN dbo.orders_test ON dbo.new_table_id.id = dbo.orders_test.customer_id
2025-09-23 10:36:00,343 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
mee (age 31) bought product1 for 2
2025-09-23 10:36:00,455 INFO sqlalchemy.engine.Engine ROLLBACK


AttributeError: 'NoneType' object has no attribute 'product'

In [ ]:
from datetime import datetime

    


In [ ]:
import os
import struct
import urllib
from sqlmodel import SQLModel, Field, Session, select, Relationship
from sqlalchemy import create_engine, event, Column
from azure.identity import DefaultAzureCredential, ClientSecretCredential
import uuid
from sqlalchemy.dialects.mssql import UNIQUEIDENTIFIER
from datetime import datetime
from pydantic import BaseModel
from typing import Optional
from uuid import UUID
# --- Azure creds ---
driver_name = "{ODBC Driver 18 for SQL Server}"
server_name = os.getenv("SERVER")      # e.g. "xxx.datawarehouse.fabric.microsoft.com"
database_name = os.getenv("DATABASE")

# --- Connection string (NO auth here, token comes later) ---
connection_string = (
    "Driver={};Server=tcp:{},1433;Database={};Encrypt=yes;"
    "TrustServerCertificate=no;Connection Timeout=30"
).format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
odbc_str = f"mssql+pyodbc:///?odbc_connect={params}"

# --- Get SQLAlchemy engine ---
engine = create_engine(odbc_str, echo=True)

# --- Token injection hook ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
credential = ClientSecretCredential(tenant_id=azure_tenant_id,client_id=azure_client_id,client_secret=azure_client_secret)  # or ClientSecretCredential if you prefer

@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    print("🔑 Requesting new Entra ID token for SQL Server...")
    token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("utf-16-le")
    token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256
    cparams["attrs_before"] = {SQL_COPT_SS_ACCESS_TOKEN: token_struct}

# --- SQLModel tables ---

schema_name = "portfolio_dev"
class PortfolioProject(SQLModel, table=True):
    __tablename__ = "PortfolioProject"
    __table_args__ = {"schema": schema_name}
    prosjekt_sk_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    prosjekt_id: uuid.UUID
    navn: str | None = None
    oppstart: datetime | None = None
    avdeling: str | None = None
    kontaktpersoner: str | None = None
    epost_kontakt: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None


class DigitaliseringStrategi(SQLModel, table=True):
    __tablename__ = "DigitaliseringStrategi"
    __table_args__ = {"schema": schema_name}

    digitalisering_strategi_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    sammenheng_digital_strategi: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Finansiering(SQLModel, table=True):
    __tablename__ = "Finansering"
    __table_args__ = {"schema": schema_name}

    finansering_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    potensiell_finansering: float | None = None
    mnd_verk: float | None = None
    vedtatt_tildeling: float | None = None
    prognose_innmeldt: float | None = None
    prognose_tildelt: float | None = None
    tentatitv_forpliktelse: float | None = None
    estimert_budsjettbehov: float | None = None
    usikkerhet_estimat: str | None = None
    risiko_av_estimat_tall: float | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Fremskritt(SQLModel, table=True):
    __tablename__ = "Fremskritt"
    __table_args__ = {"schema": schema_name}

    fremskritt_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    fremskritt: str | None = None
    fase: str | None = None
    planlagt_ferdig: datetime | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Problemstilling(SQLModel, table=True):
    __tablename__ = "Problemstilling"
    __table_args__ = {"schema": schema_name}

    problem_stilling_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    problem: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Rapportering(SQLModel, table=True):
    __tablename__ = "Rapportering"
    __table_args__ = {"schema": schema_name}

    rapportering_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    email_rapportering: str | None = None
    viktige_endringer_kommentar: str | None = None
    konsulent_anskaffet: str | None = None
    konsulent_anskaffet_kommentar: str | None = None
    risiko_rapportert: str | None = None
    risiko_rapportert_begrunnet: str | None = None
    avhengigheter_rapportert: str | None = None
    risiko_rapportert_tall: float | None = None
    viktige_endringer: float | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )
class Resursbehov(SQLModel, table=True):
    __tablename__ = "Ressursbehov"
    __table_args__ = {"schema": schema_name}

    ressursbehov_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    estimert_budsjet_forklaring: str | None = None
    estimert_budsjet_behov: float | None = None
    antall_mandsverk_intern: float | None = None
    antall_mandsverk_ekstern: float | None = None
    antall_mandsverk_ekstern_betalt: float | None = None
    risiko_av_estimat: str | None = None
    risiko_av_estimat_tall: float | None = None
    kompetanse_som_trengs: str | None = None
    kompetanse_tilgjengelig: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Risikovurdering(SQLModel, table=True):
    __tablename__ = "Risikovurdering"
    __table_args__ = {"schema": schema_name}

    risiko_vurdering_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    vurdering: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Samarabeid(SQLModel, table=True):
    __tablename__ = "Samarbeid"
    __table_args__ = {"schema": schema_name}

    samarbeid_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    samarbeid_intern: str | None = None
    samarbeid_eksternt: str | None = None
    avhengigheter_andre: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Tiltak(SQLModel, table=True):
    __tablename__ = "Tiltak"
    __table_args__ = {"schema": schema_name}

    tiltak_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    tiltak_beskrivelse: str | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class UnderstøtteTildelingsbrev(SQLModel, table=True):
    __tablename__ = "UnderstøtteTildelingsbrev"
    __table_args__ = {"schema": schema_name}

    strategisk_forankring_id: uuid.UUID = Field(
            default_factory=uuid.uuid4,
            sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
        )
    mål_1_beskrivelse: str | None = None
    mål_2_beskrivelse: str | None = None
    mål_3_beskrivelse: str | None = None
    mål_1: float | None = None
    mål_2: float | None = None
    mål_3: float | None = None
    sist_endret: datetime | None = None
    er_gjeldende: bool | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class Vurdering(SQLModel, table=True):
    __tablename__ = "Vurdering"
    __table_args__ = {"schema": schema_name}

    vurdering_id: uuid.UUID = Field(
        default_factory=uuid.uuid4,
        sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
    )
    gruppe: str | None = None
    pulje: int | None = None
    risiko_vurdering: str | None = None
    sist_endret: datetime
    er_gjeldende: bool | None = None
    mscw: str | None = None
    prosjekt_id : uuid.UUID = Field(
        foreign_key="portfolio.PortfolioProject.prosjekt_id",  # 👈 link to users
    )

class ProjectData(BaseModel):
    prosjekt_id: UUID
    date_modified: Optional[datetime]
    navn_tiltak: Optional[str]
    kontaktperson: Optional[str]
    avdeling: Optional[str]
    fase_tiltak: Optional[str]
    oppstart_tid: Optional[datetime]
    ferdig_tid: Optional[datetime]
    samarbeid_intern: Optional[str]
    samarbeid_eksternt: Optional[str]
    avhengigheter_andre: Optional[str]
    problemstilling: Optional[str]
    beskrivelse: Optional[str]
    risiko: Optional[str]
    mål_1_tildelingsbrevet: Optional[str]
    mål_2_tildelingsbrevet: Optional[str]
    mål_3_tildelingsbrevet: Optional[str]
    kompetanse_behov: Optional[str]
    kompetanse_internt: Optional[str]
    månedsverk_interne: Optional[int]
    månedsverk_eksterne: Optional[int]
    månedsverk_eksterne_2025: Optional[int]
    estimert_behov_utover_driftsrammen: Optional[float]
    estimert_behov_forklaring: Optional[str]
    hvor_sikkert_estimatene: Optional[int]
    sammenheng_med_digitaliseringsstrategien_mm: Optional[str]
    eier_epost: str

convert_list = {"prosjekt_id": PortfolioProject.prosjekt_id,
"date_modified": PortfolioProject.sist_endret,
"navn_tiltak" : PortfolioProject.navn,
"kontaktperson"	: PortfolioProject.kontaktpersoner,
"avdeling" : PortfolioProject.avdeling,
"fase_tiltak": Fremskritt.fase,
"oppstart_tid": PortfolioProject.oppstart,
"ferdig_tid": Fremskritt.planlagt_ferdig,
"samarbeid_intern": Samarabeid.samarbeid_intern,
"samarbeid_eksternt": Samarabeid.samarbeid_eksternt,
"avhengigheter_andre": Samarabeid.avhengigheter_andre,
"problemstilling": Problemstilling.problem,
"beskrivelse": Tiltak.tiltak_beskrivelse,
"risiko": Risikovurdering.vurdering,
"mål_1_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_1_beskrivelse,
"mål_2_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_2_beskrivelse,
"mål_3_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_3_beskrivelse,
"kompetanse_behov": Resursbehov.kompetanse_som_trengs,
"kompetanse_internt": Resursbehov.kompetanse_tilgjengelig,
"månedsverk_interne": Resursbehov.antall_mandsverk_intern,
"månedsverk_eksterne": Resursbehov.antall_mandsverk_ekstern,
"månedsverk_eksterne_2025": Resursbehov.antall_mandsverk_ekstern_betalt,
"estimert_behov_utover_driftsrammen": Resursbehov.estimert_budsjet_behov,
"estimert_behov_forklaring": Resursbehov.estimert_budsjet_forklaring,	
"hvor_sikkert_estimatene": Resursbehov.risiko_av_estimat_tall,
"sammenheng_med_digitaliseringsstrategien_mm": DigitaliseringStrategi.sammenheng_digital_strategi,
"eier_epost": PortfolioProject.epost_kontakt}

field_to_table_col = {"prosjekt_id": (PortfolioProject,PortfolioProject.prosjekt_id),
"date_modified": (PortfolioProject,PortfolioProject.sist_endret),
"navn_tiltak" : (PortfolioProject,PortfolioProject.navn),
"kontaktperson"	: (PortfolioProject,PortfolioProject.kontaktpersoner),
"avdeling" : (PortfolioProject,PortfolioProject.avdeling),
"fase_tiltak": (Fremskritt,Fremskritt.fase),
"oppstart_tid": (PortfolioProject,PortfolioProject.oppstart),
"ferdig_tid": (Fremskritt,Fremskritt.planlagt_ferdig),
"samarbeid_intern": (Samarabeid,Samarabeid.samarbeid_intern),
"samarbeid_eksternt": (Samarabeid,Samarabeid.samarbeid_eksternt),
"avhengigheter_andre": (Samarabeid,Samarabeid.avhengigheter_andre),
"problemstilling": (Problemstilling,Problemstilling.problem),
"beskrivelse": (Tiltak,Tiltak.tiltak_beskrivelse),
"risiko": (Risikovurdering,Risikovurdering.vurdering),
"mål_1_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_1_beskrivelse),
"mål_2_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_2_beskrivelse),
"mål_3_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_3_beskrivelse),
"kompetanse_behov": (Resursbehov,Resursbehov.kompetanse_som_trengs),
"kompetanse_internt": (Resursbehov,Resursbehov.kompetanse_tilgjengelig),
"månedsverk_interne": (Resursbehov,Resursbehov.antall_mandsverk_intern),
"månedsverk_eksterne": (Resursbehov,Resursbehov.antall_mandsverk_ekstern),
"månedsverk_eksterne_2025": (Resursbehov,Resursbehov.antall_mandsverk_ekstern_betalt),
"estimert_behov_utover_driftsrammen": (Resursbehov,Resursbehov.estimert_budsjet_behov),
"estimert_behov_forklaring": (Resursbehov,Resursbehov.estimert_budsjet_forklaring),    
"hvor_sikkert_estimatene": (Resursbehov,Resursbehov.risiko_av_estimat_tall),
"sammenheng_med_digitaliseringsstrategien_mm": (DigitaliseringStrategi,DigitaliseringStrategi.sammenheng_digital_strategi),
"eier_epost": (PortfolioProject,PortfolioProject.epost_kontakt)}

table_pk_map = {
    PortfolioProject: "prosjekt_sk_id",
    DigitaliseringStrategi: "digitalisering_strategi_id",
    Finansiering: "finansering_id",
    Fremskritt: "fremskritt_id",
    Problemstilling: "problem_stilling_id",
    Rapportering: "rapportering_id",
    Resursbehov: "ressursbehov_id",
    Risikovurdering: "risiko_vurdering_id",
    Samarabeid: "samarbeid_id",
    Tiltak: "tiltak_id",
    UnderstøtteTildelingsbrev: "strategisk_forankring_id",
    Vurdering: "vurdering_id",
}


def get_project_data(session, email: str):
    stmt = (
        select(*convert_list.values())
        .join(Fremskritt, Fremskritt.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Samarabeid, Samarabeid.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Problemstilling, Problemstilling.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Tiltak, Tiltak.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Risikovurdering, Risikovurdering.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(UnderstøtteTildelingsbrev, UnderstøtteTildelingsbrev.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Resursbehov, Resursbehov.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(DigitaliseringStrategi, DigitaliseringStrategi.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .where(
            PortfolioProject.er_gjeldende == True,
            PortfolioProject.epost_kontakt == email,
            DigitaliseringStrategi.er_gjeldende == True,
            Fremskritt.er_gjeldende == True,
            Problemstilling.er_gjeldende == True,
            Resursbehov.er_gjeldende == True,
            Risikovurdering.er_gjeldende == True,
            Samarabeid.er_gjeldende == True,
            Tiltak.er_gjeldende == True,
            UnderstøtteTildelingsbrev.er_gjeldende == True
        )
    )
    results = session.exec(stmt).all()
    projects = [
        ProjectData(
            **{
                alias: row[i] 
                for i, (alias, col) in enumerate(convert_list.items())
            }
        )
        for row in results
    ]

    return projects




from collections import Counter
from typing import List, Any
from uuid import UUID

def diff_projects(
    original: List[Any],
    edited: List[Any],
    key_field: str = "prosjekt_id",
    include_change_type: bool = False,
) -> List[dict]:
    """
    Compare two lists of Pydantic/SQLModel objects by the given key_field (default 'prosjekt_id').

    Returns a list of dicts:
      - For modified rows: {"prosjekt_id": <UUID>, "changes": {field: {"old":..., "new":...}}, (optional) "change_type": "modified"}
      - For newly added rows: {"prosjekt_id": <UUID>, "changes": {...}, (optional) "change_type": "added"}
      - For removed rows (present in original but not in edited): {"prosjekt_id": <UUID>, "changes": None, (optional) "change_type": "removed", "old_record": <original_obj>}

    Notes:
      - Compares fields present in either original or edited model (union).
      - If duplicate prosjekt_id values exist in either list, raises ValueError to avoid ambiguous matches.
    """

    # === 1) sanity: check duplicates ===
    def _find_dupes(lst):
        keys = [getattr(x, key_field) for x in lst]
        return [k for k, c in Counter(keys).items() if c > 1]

    dup_orig = _find_dupes(original)
    dup_edit = _find_dupes(edited)
    if dup_orig or dup_edit:
        raise ValueError(
            f"Duplicate {key_field} found in input lists. "
            f"original duplicates: {dup_orig}, edited duplicates: {dup_edit}"
        )

    # === 2) build maps by prosjekt_id ===
    orig_map = {getattr(o, key_field): o for o in original}
    edit_map = {getattr(e, key_field): e for e in edited}

    results = []

    # === 3) handle added and modified (iterate edited) ===
    for pid, e in edit_map.items():
        o = orig_map.get(pid)
        # determine field names to compare: union of model_fields if available, else __dict__ keys
        keys = set()
        if hasattr(e, "model_fields"):
            keys.update(e.model_fields.keys())
        else:
            keys.update(k for k in vars(e).keys() if not k.startswith("_"))

        if o is not None:
            if hasattr(o, "model_fields"):
                keys.update(o.model_fields.keys())
            else:
                keys.update(k for k in vars(o).keys() if not k.startswith("_"))

        # compute diffs
        diffs = {}
        for field in keys:
            old = getattr(o, field, None) if o is not None else None
            new = getattr(e, field, None)
            if old != new:
                diffs[field] = {"old": old, "new": new}

        if diffs:
            entry = {"prosjekt_id": pid, "changes": diffs}
            if include_change_type:
                entry["change_type"] = "modified" if o is not None else "added"
            results.append(entry)

    # === 4) handle removed ===
    for pid, o in orig_map.items():
        if pid not in edit_map:
            entry = {"prosjekt_id": pid, "changes": None}
            if include_change_type:
                entry["change_type"] = "removed"
                entry["old_record"] = o
            results.append(entry)

    return results


from collections import defaultdict
def apply_changes(diffs, session):
    for diff in diffs:
        prosjekt_id = diff["prosjekt_id"]
        changes = diff["changes"]

        # 1️⃣ Group changes by table
        changes_by_table = defaultdict(dict)
        for field, change in changes.items():
            table_cls, col_attr = field_to_table_col[field]
            changes_by_table[table_cls][field] = change

        # 2️⃣ Apply updates per table
        for table_cls, table_changes in changes_by_table.items():
            stmt = select(table_cls).where(
                table_cls.prosjekt_id == prosjekt_id,
                table_cls.er_gjeldende == True
            )
            current = session.exec(stmt).one_or_none()
            if not current:
                print(f"⚠️ No current row found for {table_cls.__tablename__}, prosjekt_id={prosjekt_id}")
                continue

            # Mark old row as not current
            current.er_gjeldende = False
            session.add(current)

            # Copy current row values
            new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
            pk_col = table_pk_map[table_cls]
            new_row_data.pop(pk_col, None)  # remove surrogate key
            new_row_data[pk_col] = str(uuid.uuid4())  # or let DB autogenerate
            new_row_data["sist_endret"] = datetime.utcnow()
            new_row_data["er_gjeldende"] = True

            # Apply all changes for this table
            for field, change in table_changes.items():
                _, col_attr = field_to_table_col[field]
                col_name = col_attr.key  # SQLAlchemy column name
                new_row_data[col_name] = change["new"]

            # Insert new row
            new_row = table_cls(**new_row_data)
            session.add(new_row)

    session.commit()


c:\Users\IGNTOR\OneDrive - Digitaliseringsdirektoratet\Dokumenter\GitHub\projectportfolio\.venv\Lib\site-packages\sqlmodel\main.py:641: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.PortfolioProject, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


InvalidRequestError: Table 'portfolio_dev.PortfolioProject' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
# def diff_projects(original, edited):
#     changes = []
#     for o, e in zip(original, edited):
#         diffs = {}
#         for field in e.model_fields.keys():
#             if getattr(o, field) != getattr(e, field):
#                 diffs[field] = {
#                     "old": getattr(o, field),
#                     "new": getattr(e, field)
#                 }
#         if diffs:
#             changes.append({"prosjekt_id": e.prosjekt_id, "changes": diffs})
#     return changes

# # --- Usage example ---
# with Session(engine) as session:
#     projects = get_project_data(session, email="jonhakon.odd@digdir.no")
#     for project in projects:
#         for key, values in project.items():
#             print(f"{key}: {values}")

In [69]:
from collections import Counter
from typing import List, Any
from uuid import UUID

def diff_projects(
    original: List[Any],
    edited: List[Any],
    key_field: str = "prosjekt_id",
    include_change_type: bool = False,
) -> List[dict]:
    """
    Compare two lists of Pydantic/SQLModel objects by the given key_field (default 'prosjekt_id').

    Returns a list of dicts:
      - For modified rows: {"prosjekt_id": <UUID>, "changes": {field: {"old":..., "new":...}}, (optional) "change_type": "modified"}
      - For newly added rows: {"prosjekt_id": <UUID>, "changes": {...}, (optional) "change_type": "added"}
      - For removed rows (present in original but not in edited): {"prosjekt_id": <UUID>, "changes": None, (optional) "change_type": "removed", "old_record": <original_obj>}

    Notes:
      - Compares fields present in either original or edited model (union).
      - If duplicate prosjekt_id values exist in either list, raises ValueError to avoid ambiguous matches.
    """

    # === 1) sanity: check duplicates ===
    def _find_dupes(lst):
        keys = [getattr(x, key_field) for x in lst]
        return [k for k, c in Counter(keys).items() if c > 1]

    dup_orig = _find_dupes(original)
    dup_edit = _find_dupes(edited)
    if dup_orig or dup_edit:
        raise ValueError(
            f"Duplicate {key_field} found in input lists. "
            f"original duplicates: {dup_orig}, edited duplicates: {dup_edit}"
        )

    # === 2) build maps by prosjekt_id ===
    orig_map = {getattr(o, key_field): o for o in original}
    edit_map = {getattr(e, key_field): e for e in edited}

    results = []

    # === 3) handle added and modified (iterate edited) ===
    for pid, e in edit_map.items():
        o = orig_map.get(pid)
        # determine field names to compare: union of model_fields if available, else __dict__ keys
        keys = set()
        if hasattr(e, "model_fields"):
            keys.update(e.model_fields.keys())
        else:
            keys.update(k for k in vars(e).keys() if not k.startswith("_"))

        if o is not None:
            if hasattr(o, "model_fields"):
                keys.update(o.model_fields.keys())
            else:
                keys.update(k for k in vars(o).keys() if not k.startswith("_"))

        # compute diffs
        diffs = {}
        for field in keys:
            old = getattr(o, field, None) if o is not None else None
            new = getattr(e, field, None)
            if old != new:
                diffs[field] = {"old": old, "new": new}

        if diffs:
            entry = {"prosjekt_id": pid, "changes": diffs}
            if include_change_type:
                entry["change_type"] = "modified" if o is not None else "added"
            results.append(entry)

    # === 4) handle removed ===
    for pid, o in orig_map.items():
        if pid not in edit_map:
            entry = {"prosjekt_id": pid, "changes": None}
            if include_change_type:
                entry["change_type"] = "removed"
                entry["old_record"] = o
            results.append(entry)

    return results


In [5]:
email = "jonhakon.odd@digdir.no"
with Session(engine) as session:
#     projects = get_project_data(session, email="jonhakon.odd@digdir.no")
    results = get_project_data(session, email=email)

🔑 Requesting new Entra ID token for SQL Server...
2025-09-24 13:39:33,722 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-09-24 13:39:33,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-24 13:39:33,729 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-09-24 13:39:33,730 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
2025-09-24 13:39:33,759 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-09-24 13:39:33,760 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
2025-09-24 13:39:33,767 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-09-24 13:39:33,768 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ()
2025-09-24 13:39:33,819 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 13:39:33,832 INFO sqlalchemy.engine.Engine SELECT portfolio_dev.[PortfolioProject].prosjekt_id, portfolio_dev.[Por

In [ ]:
from copy import deepcopy
email = "jonhakon.odd@digdir.no"
original = get_project_data(session,email=email)
edited = deepcopy(original)
edited[0].navn_tiltak = "New Name"
edited[0].kontaktperson = "New Contact"
diffs = diff_projects(original, edited)


2025-09-24 13:46:26,843 INFO sqlalchemy.engine.Engine SELECT portfolio_dev.[PortfolioProject].prosjekt_id, portfolio_dev.[PortfolioProject].sist_endret, portfolio_dev.[PortfolioProject].navn, portfolio_dev.[PortfolioProject].kontaktpersoner, portfolio_dev.[PortfolioProject].avdeling, portfolio_dev.[Fremskritt].fase, portfolio_dev.[PortfolioProject].oppstart, portfolio_dev.[Fremskritt].planlagt_ferdig, portfolio_dev.[Samarbeid].samarbeid_intern, portfolio_dev.[Samarbeid].samarbeid_eksternt, portfolio_dev.[Samarbeid].avhengigheter_andre, portfolio_dev.[Problemstilling].problem, portfolio_dev.[Tiltak].tiltak_beskrivelse, portfolio_dev.[Risikovurdering].vurdering, portfolio_dev.[UnderstøtteTildelingsbrev].[mål_1_beskrivelse], portfolio_dev.[UnderstøtteTildelingsbrev].[mål_2_beskrivelse], portfolio_dev.[UnderstøtteTildelingsbrev].[mål_3_beskrivelse], portfolio_dev.[Ressursbehov].kompetanse_som_trengs, portfolio_dev.[Ressursbehov].kompetanse_tilgjengelig, portfolio_dev.[Ressursbehov].antall_

C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\4206015110.py:5: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field in e.model_fields.keys():


In [72]:
original = get_project_data(session,email=email)

2025-09-24 14:37:05,940 INFO sqlalchemy.engine.Engine SELECT portfolio_dev.[PortfolioProject].prosjekt_id, portfolio_dev.[PortfolioProject].sist_endret, portfolio_dev.[PortfolioProject].navn, portfolio_dev.[PortfolioProject].kontaktpersoner, portfolio_dev.[PortfolioProject].avdeling, portfolio_dev.[Fremskritt].fase, portfolio_dev.[PortfolioProject].oppstart, portfolio_dev.[Fremskritt].planlagt_ferdig, portfolio_dev.[Samarbeid].samarbeid_intern, portfolio_dev.[Samarbeid].samarbeid_eksternt, portfolio_dev.[Samarbeid].avhengigheter_andre, portfolio_dev.[Problemstilling].problem, portfolio_dev.[Tiltak].tiltak_beskrivelse, portfolio_dev.[Risikovurdering].vurdering, portfolio_dev.[UnderstøtteTildelingsbrev].[mål_1_beskrivelse], portfolio_dev.[UnderstøtteTildelingsbrev].[mål_2_beskrivelse], portfolio_dev.[UnderstøtteTildelingsbrev].[mål_3_beskrivelse], portfolio_dev.[Ressursbehov].kompetanse_som_trengs, portfolio_dev.[Ressursbehov].kompetanse_tilgjengelig, portfolio_dev.[Ressursbehov].antall_

In [73]:
original

[ProjectData(prosjekt_id=UUID('e5200876-e373-4b5e-a8b6-a97f6faf75c4'), date_modified=datetime.datetime(2024, 12, 6, 12, 12, 40), navn_tiltak='Forenklet onboarding og økt bruk av fellesløsnigner ', kontaktperson='Jon Håkon Odd', avdeling='FEL', fase_tiltak='Konsept', oppstart_tid=datetime.datetime(2025, 1, 1, 0, 0), ferdig_tid=None, samarbeid_intern='BOD', samarbeid_eksternt=None, avhengigheter_andre='Digdir MO og Mottaksprosjekt ', problemstilling='Det er i områdegjennomgangen blant annet pekt på at det er behov for mer operativ og målrettet infomrasjon om fellesløsningen og veikart for utvikling. I dag er det ulike onboardnigsprosesser for bruk av fellesløsningene. Det gjør det vanskelig for eksisterende og potensielle tjenesteiere og forstå hvilke fellesløsninger som er relvante og være tilstrekkelig selvbetjente. Videre er det det behov for å bedre informasjon om veikart og utvikling knyttet til produktene og hvordan de tas i bruk. ', beskrivelse='Enklere og enhetlig onboarding (vei

In [76]:
edited

[ProjectData(prosjekt_id=UUID('c160766f-6744-4629-8b65-3f8de545356f'), date_modified=datetime.datetime(2024, 12, 5, 11, 54, 54), navn_tiltak='New Name', kontaktperson='New Contact', avdeling='FEL', fase_tiltak='Konsept', oppstart_tid=datetime.datetime(2025, 1, 1, 0, 0), ferdig_tid=None, samarbeid_intern='BOD, VIS', samarbeid_eksternt=None, avhengigheter_andre=None, problemstilling='Dagens håndterelse av utfaktureringen av fellestjenestene ID-Porten, Digital Postkasse til Innbyggere, eSignering, eInnsyn og Maskinporten er i stor grad en personavhengig og tidkrevende prosess. ID-Porten, eInnsyn og Maskinporten gjøres manuelt, hvor DPI og eSignering gjøres semi-automatisk ved hjelp av en lokal database. Kunder har svært også begrenset med muligheter til å gjøre endringer og/eller overvåke forbruket sitt uten å kontakte oss. Med nye finansieringsmodeller på horisonten som innebærer økt brukerfinansiering av fellestjenestene blir det behov for tilpasninger og skalering som dagens system ikk

In [74]:

diffs = diff_projects(original, edited)

C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\347132754.py:48: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if hasattr(e, "model_fields"):
C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\347132754.py:49: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  keys.update(e.model_fields.keys())
C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\347132754.py:54: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if hasattr(o, "model_fields"):
C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\

In [75]:
diffs

[{'prosjekt_id': UUID('c160766f-6744-4629-8b65-3f8de545356f'),
  'changes': {'navn_tiltak': {'old': None, 'new': 'New Name'},
   'oppstart_tid': {'old': None, 'new': datetime.datetime(2025, 1, 1, 0, 0)},
   'prosjekt_id': {'old': None,
    'new': UUID('c160766f-6744-4629-8b65-3f8de545356f')},
   'kompetanse_internt': {'old': None,
    'new': 'Ja, men ikke overførbare da det vil påvirke produksjonen i for stor grad'},
   'mål_2_tildelingsbrevet': {'old': None, 'new': 'Noe bidrag til mål'},
   'eier_epost': {'old': None, 'new': 'jonhakon.odd@digdir.no'},
   'månedsverk_eksterne': {'old': None, 'new': 10},
   'estimert_behov_forklaring': {'old': None, 'new': 'Ev. forklaring'},
   'avdeling': {'old': None, 'new': 'FEL'},
   'risiko': {'old': None,
    'new': 'Svekket brukerfinansering og kundetilfredshet ved bruk av fellestjenestene. '},
   'månedsverk_interne': {'old': None, 'new': 20},
   'problemstilling': {'old': None,
    'new': 'Dagens håndterelse av utfaktureringen av fellestjeneste

In [ ]:
field_to_table_col = {"prosjekt_id": (PortfolioProject,PortfolioProject.prosjekt_id),
"date_modified": (PortfolioProject,PortfolioProject.sist_endret),
"navn_tiltak" : (PortfolioProject,PortfolioProject.navn),
"kontaktperson"	: (PortfolioProject,PortfolioProject.kontaktpersoner),
"avdeling" : (PortfolioProject,PortfolioProject.avdeling),
"fase_tiltak": (Fremskritt,Fremskritt.fase),
"oppstart_tid": (PortfolioProject,PortfolioProject.oppstart),
"ferdig_tid": (Fremskritt,Fremskritt.planlagt_ferdig),
"samarbeid_intern": (Samarabeid,Samarabeid.samarbeid_intern),
"samarbeid_eksternt": (Samarabeid,Samarabeid.samarbeid_eksternt),
"avhengigheter_andre": (Samarabeid,Samarabeid.avhengigheter_andre),
"problemstilling": (Problemstilling,Problemstilling.problem),
"beskrivelse": (Tiltak,Tiltak.tiltak_beskrivelse),
"risiko": (Risikovurdering,Risikovurdering.vurdering),
"mål_1_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_1_beskrivelse),
"mål_2_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_2_beskrivelse),
"mål_3_tildelingsbrevet": (UnderstøtteTildelingsbrev,UnderstøtteTildelingsbrev.mål_3_beskrivelse),
"kompetanse_behov": (Resursbehov,Resursbehov.kompetanse_som_trengs),
"kompetanse_internt": (Resursbehov,Resursbehov.kompetanse_tilgjengelig),
"månedsverk_interne": (Resursbehov,Resursbehov.antall_mandsverk_intern),
"månedsverk_eksterne": (Resursbehov,Resursbehov.antall_mandsverk_ekstern),
"månedsverk_eksterne_2025": (Resursbehov,Resursbehov.antall_mandsverk_ekstern_betalt),
"estimert_behov_utover_driftsrammen": (Resursbehov,Resursbehov.estimert_budsjet_behov),
"estimert_behov_forklaring": (Resursbehov,Resursbehov.estimert_budsjet_forklaring),    
"hvor_sikkert_estimatene": (Resursbehov,Resursbehov.risiko_av_estimat_tall),
"sammenheng_med_digitaliseringsstrategien_mm": (DigitaliseringStrategi,DigitaliseringStrategi.sammenheng_digital_strategi),
"eier_epost": (PortfolioProject,PortfolioProject.epost_kontakt)}

table_pk_map = {
    PortfolioProject: "prosjekt_sk_id",
    DigitaliseringStrategi: "digitalisering_strategi_id",
    Finansiering: "finansering_id",
    Fremskritt: "fremskritt_id",
    Problemstilling: "problem_stilling_id",
    Rapportering: "rapportering_id",
    Resursbehov: "ressursbehov_id",
    Risikovurdering: "risiko_vurdering_id",
    Samarabeid: "samarbeid_id",
    Tiltak: "tiltak_id",
    UnderstøtteTildelingsbrev: "strategisk_forankring_id",
    Vurdering: "vurdering_id",
}



In [59]:
table_pk_map = {
    PortfolioProject: "prosjekt_sk_id",
    DigitaliseringStrategi: "digitalisering_strategi_id",
    Finansiering: "finansering_id",
    Fremskritt: "fremskritt_id",
    Problemstilling: "problem_stilling_id",
    Rapportering: "rapportering_id",
    Resursbehov: "ressursbehov_id",
    Risikovurdering: "risiko_vurdering_id",
    Samarabeid: "samarbeid_id",
    Tiltak: "tiltak_id",
    UnderstøtteTildelingsbrev: "strategisk_forankring_id",
    Vurdering: "vurdering_id",
}

In [60]:
table_pk_map[table_cls]

'prosjekt_sk_id'

In [61]:
from collections import defaultdict
def apply_changes(diffs, session):
    for diff in diffs:
        prosjekt_id = diff["prosjekt_id"]
        for field, change in diff["changes"].items():
            table_cls, col_attr = field_to_table_col[field]

            # 1. Get current row
            stmt = select(table_cls).where(
                table_cls.prosjekt_id == prosjekt_id,
                table_cls.er_gjeldende == True
            )
            current = session.exec(stmt).one_or_none()
            if not current:
                print(f"⚠️ No current row found for {table_cls.__tablename__}, prosjekt_id={prosjekt_id}")
                continue

            # 2. Mark old row as not current
            current.er_gjeldende = False
            session.add(current)

            # 3. Insert new row

            new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
            pk_col = table_pk_map[table_cls]
            new_row_data.pop(pk_col, None)  # remove surrogate key
            new_row_data[pk_col] = str(uuid.uuid4())  # or let DB autogenerate
            new_row_data["sist_endret"] = datetime.utcnow()
            new_row_data["er_gjeldende"] = True
            new_row_data[col_attr.name] = change["new"]

            new_row = table_cls(**new_row_data)
            session.add(new_row)

    session.commit()

In [62]:
from collections import defaultdict
def apply_changes(diffs, session):
    for diff in diffs:
        prosjekt_id = diff["prosjekt_id"]
        changes = diff["changes"]

        # 1️⃣ Group changes by table
        changes_by_table = defaultdict(dict)
        for field, change in changes.items():
            table_cls, col_attr = field_to_table_col[field]
            changes_by_table[table_cls][field] = change

        # 2️⃣ Apply updates per table
        for table_cls, table_changes in changes_by_table.items():
            stmt = select(table_cls).where(
                table_cls.prosjekt_id == prosjekt_id,
                table_cls.er_gjeldende == True
            )
            current = session.exec(stmt).one_or_none()
            if not current:
                print(f"⚠️ No current row found for {table_cls.__tablename__}, prosjekt_id={prosjekt_id}")
                continue

            # Mark old row as not current
            current.er_gjeldende = False
            session.add(current)

            # Copy current row values
            new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
            pk_col = table_pk_map[table_cls]
            new_row_data.pop(pk_col, None)  # remove surrogate key
            new_row_data[pk_col] = str(uuid.uuid4())  # or let DB autogenerate
            new_row_data["sist_endret"] = datetime.utcnow()
            new_row_data["er_gjeldende"] = True

            # Apply all changes for this table
            for field, change in table_changes.items():
                _, col_attr = field_to_table_col[field]
                col_name = col_attr.key  # SQLAlchemy column name
                new_row_data[col_name] = change["new"]

            # Insert new row
            new_row = table_cls(**new_row_data)
            session.add(new_row)

    session.commit()

In [64]:
with Session(engine) as session:
    apply_changes(diffs, session)

🔑 Requesting new Entra ID token for SQL Server...
2025-09-24 14:27:27,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 14:27:27,863 INFO sqlalchemy.engine.Engine SELECT portfolio_dev.[PortfolioProject].prosjekt_sk_id, portfolio_dev.[PortfolioProject].prosjekt_id, portfolio_dev.[PortfolioProject].navn, portfolio_dev.[PortfolioProject].oppstart, portfolio_dev.[PortfolioProject].avdeling, portfolio_dev.[PortfolioProject].kontaktpersoner, portfolio_dev.[PortfolioProject].epost_kontakt, portfolio_dev.[PortfolioProject].sist_endret, portfolio_dev.[PortfolioProject].er_gjeldende 
FROM portfolio_dev.[PortfolioProject] 
WHERE portfolio_dev.[PortfolioProject].prosjekt_id = ? AND portfolio_dev.[PortfolioProject].er_gjeldende = 1
2025-09-24 14:27:27,864 INFO sqlalchemy.engine.Engine [cached since 2451s ago] (UUID('c160766f-6744-4629-8b65-3f8de545356f'),)
2025-09-24 14:27:34,802 INFO sqlalchemy.engine.Engine UPDATE portfolio_dev.[PortfolioProject] SET er_gjeldende=? WHERE portfolio_dev

C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\571612372.py:29: DeprecationWarning: 
        🚨 `obj.dict()` was deprecated in SQLModel 0.0.14, you should
        instead use `obj.model_dump()`.
        
  new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\571612372.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  new_row_data["sist_endret"] = datetime.utcnow()


2025-09-24 14:27:39,790 INFO sqlalchemy.engine.Engine INSERT INTO portfolio_dev.[PortfolioProject] (prosjekt_sk_id, prosjekt_id, navn, oppstart, avdeling, kontaktpersoner, epost_kontakt, sist_endret, er_gjeldende) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-09-24 14:27:39,791 INFO sqlalchemy.engine.Engine [cached since 2085s ago] ('cb3700c3-9dcd-4627-a0db-389a240e91a2', UUID('c160766f-6744-4629-8b65-3f8de545356f'), 'New Name', datetime.datetime(2025, 1, 1, 0, 0), 'FEL', 'New Contact', 'jonhakon.odd@digdir.no', datetime.datetime(2025, 9, 24, 12, 27, 34, 800058), 1)
2025-09-24 14:27:40,062 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
for diff in diffs:
    prosjekt_id = diff["prosjekt_id"]
    changes = diff["changes"]

    # 1️⃣ Group changes by table
    changes_by_table = defaultdict(dict)
    for field, change in changes.items():
        table_cls, col_attr = field_to_table_col[field]
        changes_by_table[table_cls][field] = change


In [49]:
for table_cls, table_changes in changes_by_table.items():
    # with Session(engine) as session:
    #     stmt = select(table_cls).where(
    #             table_cls.prosjekt_id == prosjekt_id,
    #             table_cls.er_gjeldende == True
    #         )
    #     current = session.exec(stmt).one_or_none()
    for field, change in table_changes.items():
        _, col_attr = field_to_table_col[field]
        col_name = col_attr.key
        print(col_name, change["old"], change["new"])

navn Felles utfaktureringssystem New Name
kontaktpersoner Jon Håkon Odd New Contact


In [44]:
current

PortfolioProject(oppstart=datetime.datetime(2025, 1, 1, 0, 0), prosjekt_sk_id=UUID('74dfbac8-9158-46a6-a28b-9f226d14957d'), kontaktpersoner='Jon Håkon Odd', sist_endret=datetime.datetime(2025, 9, 24, 11, 52, 34, 872522), er_gjeldende=True, prosjekt_id=UUID('c160766f-6744-4629-8b65-3f8de545356f'), navn='New Name', avdeling='FEL', epost_kontakt='jonhakon.odd@digdir.no')

In [32]:
for diff in diffs:
    
    prosjekt_id = diff["prosjekt_id"]
    for field, change in diff["changes"].items():
        table_cls, col_attr = field_to_table_col[field]

        # 1. Get current row
        stmt = select(table_cls).where(
            table_cls.prosjekt_id == prosjekt_id,
            table_cls.er_gjeldende == True
        )

In [37]:
diffs

[{'prosjekt_id': UUID('c160766f-6744-4629-8b65-3f8de545356f'),
  'changes': {'navn_tiltak': {'old': 'Felles utfaktureringssystem',
    'new': 'New Name'},
   'kontaktperson': {'old': 'Jon Håkon Odd', 'new': 'New Contact'}}}]

In [28]:
col_attr.name

'navn'

In [ ]:
with Session(engine) as session:
    current = session.exec(stmt).one_or_none()
    if not current:
        print(f"⚠️ No current row found for prosjekt_id {prosjekt_id} in table {table_cls.__tablename__}")
        continue

In [20]:

    if not current:
            print(f"⚠️ No current row found for {table_cls.__tablename__}, prosjekt_id={prosjekt_id}")
            continue

        # 2. Mark old row as not current
        current.er_gjeldende = False
        session.add(current)

        # 3. Insert new row
        new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
        new_row_data.pop("prosjekt_sk_id", None)  # remove surrogate key
        new_row_data["prosjekt_sk_id"] = str(uuid.uuid4())  # or let DB autogenerate
        new_row_data["sist_endret"] = datetime.utcnow()
        new_row_data["er_gjeldende"] = True
        new_row_data[field] = change["new"]

        new_row = table_cls(**new_row_data)
        session.add(new_row)

    session.commit()

IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [17]:
current.er_gjeldende = False
session.add(current)

In [24]:
change

{'old': 'Felles utfaktureringssystem', 'new': 'New Name'}

In [29]:
new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
new_row_data.pop("prosjekt_sk_id", None)  # remove surrogate key
new_row_data["prosjekt_sk_id"] = str(uuid.uuid4())  # or let DB autogenerate
new_row_data["sist_endret"] = datetime.utcnow()
new_row_data["er_gjeldende"] = True
new_row_data[col_attr.name] = change["new"]

new_row = table_cls(**new_row_data)

C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\479441152.py:1: DeprecationWarning: 
        🚨 `obj.dict()` was deprecated in SQLModel 0.0.14, you should
        instead use `obj.model_dump()`.
        
  new_row_data = current.dict() if hasattr(current, "dict") else current.__dict__.copy()
C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_47288\479441152.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  new_row_data["sist_endret"] = datetime.utcnow()


In [31]:
session.add(new_row)

session.commit()

2025-09-24 13:52:51,101 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 13:52:51,104 INFO sqlalchemy.engine.Engine UPDATE portfolio_dev.[PortfolioProject] SET er_gjeldende=? WHERE portfolio_dev.[PortfolioProject].prosjekt_sk_id = ?
2025-09-24 13:52:51,106 INFO sqlalchemy.engine.Engine [generated in 0.00154s] (0, UUID('19810cd1-e8e0-45c6-afc3-f2a6c6709abd'))
2025-09-24 13:52:55,206 INFO sqlalchemy.engine.Engine INSERT INTO portfolio_dev.[PortfolioProject] (prosjekt_sk_id, prosjekt_id, navn, oppstart, avdeling, kontaktpersoner, epost_kontakt, sist_endret, er_gjeldende) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-09-24 13:52:55,207 INFO sqlalchemy.engine.Engine [generated in 0.00098s] ('74dfbac8-9158-46a6-a28b-9f226d14957d', UUID('c160766f-6744-4629-8b65-3f8de545356f'), 'New Name', datetime.datetime(2025, 1, 1, 0, 0), 'FEL', 'Jon Håkon Odd', 'jonhakon.odd@digdir.no', datetime.datetime(2025, 9, 24, 11, 52, 34, 872522), 1)
2025-09-24 13:52:55,453 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
current

PortfolioProject(oppstart=datetime.datetime(2025, 1, 1, 0, 0), prosjekt_sk_id=UUID('19810cd1-e8e0-45c6-afc3-f2a6c6709abd'), kontaktpersoner='Jon Håkon Odd', sist_endret=datetime.datetime(2024, 12, 5, 11, 54, 54), er_gjeldende=True, prosjekt_id=UUID('c160766f-6744-4629-8b65-3f8de545356f'), navn='Felles utfaktureringssystem', avdeling='FEL', epost_kontakt='jonhakon.odd@digdir.no')

In [117]:
current

PortfolioProject(prosjekt_id=UUID('c160766f-6744-4629-8b65-3f8de545356f'), navn='Felles utfaktureringssystem', avdeling='FEL', epost_kontakt='jonhakon.odd@digdir.no', er_gjeldende=True, prosjekt_sk_id=UUID('19810cd1-e8e0-45c6-afc3-f2a6c6709abd'), oppstart=datetime.datetime(2025, 1, 1, 0, 0), kontaktpersoner='Jon Håkon Odd', sist_endret=datetime.datetime(2024, 12, 5, 11, 54, 54))

In [97]:
for i, values in diff[0]["changes"].items():
    print(convert_list[i])

PortfolioProject.navn


In [22]:
from sqlmodel import Session, select

def get_project_full(session: Session, email: str):
    stmt = (
        select(
            PortfolioProject,
            DigitaliseringStrategi,
            Finansiering,
            Fremskritt,
            Problemstilling,
            Rapportering,
            Resursbehov,
            Risikovurdering,
            Samarabeid,
            Tiltak,
            UnderstøtteTildelingsbrev,
            Vurdering,
        )
        .join(DigitaliseringStrategi, DigitaliseringStrategi.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Finansiering, Finansiering.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Fremskritt, Fremskritt.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Problemstilling, Problemstilling.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Rapportering, Rapportering.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Resursbehov, Resursbehov.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Risikovurdering, Risikovurdering.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Samarabeid, Samarabeid.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Tiltak, Tiltak.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(UnderstøtteTildelingsbrev, UnderstøtteTildelingsbrev.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Vurdering, Vurdering.prosjekt_id == PortfolioProject.prosjekt_id)
        .where(
            PortfolioProject.er_gjeldende == True,
            PortfolioProject.epost_kontakt == email,
            DigitaliseringStrategi.er_gjeldende == True,
            Finansiering.er_gjeldende == True,
            Fremskritt.er_gjeldende == True,
            Problemstilling.er_gjeldende == True,
            Rapportering.er_gjeldende == True,
            Resursbehov.er_gjeldende == True,
            Risikovurdering.er_gjeldende == True,
            Samarabeid.er_gjeldende == True,
            Tiltak.er_gjeldende == True,
            UnderstøtteTildelingsbrev.er_gjeldende == True,
            Vurdering.er_gjeldende == True,
        )
    )

    results = session.exec(stmt).all()
    return results

In [36]:
convert_list = {"prosjekt_id": PortfolioProject.prosjekt_id,
"Date modified": PortfolioProject.sist_endret,
"navn_tiltak" : PortfolioProject.navn,
"kontaktperson"	: PortfolioProject.kontaktpersoner,
"avdeling" : PortfolioProject.avdeling,
"fase_tiltak": Fremskritt.fase,
"oppstart_tid": PortfolioProject.oppstart,
"ferdig_tid": Fremskritt.planlagt_ferdig,
"samarbeid_intern": Samarabeid.samarbeid_intern,
"samarbeid_eksternt": Samarabeid.samarbeid_eksternt,
"avhengigheter_andre": Samarabeid.avhengigheter_andre,
"problemstilling": Problemstilling.problem,
"beskrivelse": Tiltak.tiltak_beskrivelse,
"risiko": Risikovurdering.vurdering,
"mål_1_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_1_beskrivelse,
"mål_2_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_2_beskrivelse,
"mål_3_tildelingsbrevet": UnderstøtteTildelingsbrev.mål_3_beskrivelse,
"kompetanse_behov": Resursbehov.kompetanse_som_trengs,
"kompetanse_internt": Resursbehov.kompetanse_tilgjengelig,
"månedsverk_interne": Resursbehov.antall_mandsverk_intern,
"månedsverk_eksterne": Resursbehov.antall_mandsverk_ekstern,
"månedsverk_eksterne_2025": Resursbehov.antall_mandsverk_ekstern_betalt,
"estimert_behov_utover_driftsrammen": Resursbehov.estimert_budsjet_behov,
"estimert_behov_forklaring": Resursbehov.estimert_budsjet_forklaring,	
"hvor_sikkert_estimatene": Resursbehov.risiko_av_estimat_tall,
"sammenheng_med_digitaliseringsstrategien_mm": DigitaliseringStrategi.sammenheng_digital_strategi,
"eier_epost": PortfolioProject.epost_kontakt}

In [24]:
from sqlalchemy.orm import joinedload
from sqlmodel import Session, select

def get_project_full(session: Session, email: str):
    stmt = (
        select(
            PortfolioProject,
            DigitaliseringStrategi,
            Finansiering,
            Fremskritt,
            Problemstilling,
            Rapportering,
            Resursbehov,
            Risikovurdering,
            Samarabeid,
            Tiltak,
            UnderstøtteTildelingsbrev,
            Vurdering,
        )
        .outerjoin(DigitaliseringStrategi, DigitaliseringStrategi.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Finansiering, Finansiering.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Fremskritt, Fremskritt.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Problemstilling, Problemstilling.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Rapportering, Rapportering.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Resursbehov, Resursbehov.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Risikovurdering, Risikovurdering.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Samarabeid, Samarabeid.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Tiltak, Tiltak.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(UnderstøtteTildelingsbrev, UnderstøtteTildelingsbrev.prosjekt_id == PortfolioProject.prosjekt_id)
        .outerjoin(Vurdering, Vurdering.prosjekt_id == PortfolioProject.prosjekt_id)
        .where(
            PortfolioProject.er_gjeldende == True,
            PortfolioProject.epost_kontakt == email,
        )
    )
    results = session.exec(stmt).all()
    return results


In [37]:
from sqlmodel import select

def get_project_data(session, email: str):
    # Build the SELECT dynamically from your dict
    stmt = (
        select(*convert_list.values())
        .join(Fremskritt, Fremskritt.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Samarabeid, Samarabeid.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Problemstilling, Problemstilling.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Tiltak, Tiltak.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Risikovurdering, Risikovurdering.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(UnderstøtteTildelingsbrev, UnderstøtteTildelingsbrev.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(Resursbehov, Resursbehov.prosjekt_id == PortfolioProject.prosjekt_id)
        .join(DigitaliseringStrategi, DigitaliseringStrategi.prosjekt_id == PortfolioProject.prosjekt_id)
        .where(
            PortfolioProject.er_gjeldende == True,
            PortfolioProject.epost_kontakt == email,
        )
    )
    results = session.exec(stmt).all()

    # Convert into dicts with your final keys instead of column objects
    data = [
        {alias: getattr(row, col.key, None) if hasattr(col, "key") else row[i]
         for i, (alias, col) in enumerate(convert_list.items())}
        for row in results
    ]

    return data

In [42]:
from sqlmodel import select

def get_project_data(session, email: str):
    stmt = (
        select(*convert_list.values())
        .join(Fremskritt, Fremskritt.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Samarabeid, Samarabeid.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Problemstilling, Problemstilling.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Tiltak, Tiltak.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Risikovurdering, Risikovurdering.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(UnderstøtteTildelingsbrev, UnderstøtteTildelingsbrev.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(Resursbehov, Resursbehov.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .join(DigitaliseringStrategi, DigitaliseringStrategi.prosjekt_id == PortfolioProject.prosjekt_id, isouter=True)
        .where(
            PortfolioProject.er_gjeldende == True,
            PortfolioProject.epost_kontakt == email,
            DigitaliseringStrategi.er_gjeldende == True,
            Fremskritt.er_gjeldende == True,
            Problemstilling.er_gjeldende == True,
            Resursbehov.er_gjeldende == True,
            Risikovurdering.er_gjeldende == True,
            Samarabeid.er_gjeldende == True,
            Tiltak.er_gjeldende == True,
            UnderstøtteTildelingsbrev.er_gjeldende == True
        )
    )
    results = session.exec(stmt).all()

    # Map results into dicts with your aliases
    data = [
        {
            alias: row[i]  # row is a tuple in order of convert_list.values()
            for i, (alias, col) in enumerate(convert_list.items())
        }
        for row in results
    ]

    return data

In [52]:
with Session(engine) as session:
    projects = get_project_data(session, email="jonhakon.odd@digdir.no")
    for project in projects:
        for key, values in project.items():
            print(f"{key}: {values}")

2025-09-23 15:31:29,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-23 15:31:29,996 INFO sqlalchemy.engine.Engine SELECT portfolio.[PortfolioProject].prosjekt_id, portfolio.[PortfolioProject].sist_endret, portfolio.[PortfolioProject].navn, portfolio.[PortfolioProject].kontaktpersoner, portfolio.[PortfolioProject].avdeling, portfolio.[Fremskritt].fase, portfolio.[PortfolioProject].oppstart, portfolio.[Fremskritt].planlagt_ferdig, portfolio.[Samarbeid].samarbeid_intern, portfolio.[Samarbeid].samarbeid_eksternt, portfolio.[Samarbeid].avhengigheter_andre, portfolio.[Problemstilling].problem, portfolio.[Tiltak].tiltak_beskrivelse, portfolio.[Risikovurdering].vurdering, portfolio.[UnderstøtteTildelingsbrev].[mål_1_beskrivelse], portfolio.[UnderstøtteTildelingsbrev].[mål_2_beskrivelse], portfolio.[UnderstøtteTildelingsbrev].[mål_3_beskrivelse], portfolio.[Ressursbehov].kompetanse_som_trengs, portfolio.[Ressursbehov].kompetanse_tilgjengelig, portfolio.[Ressursbehov].antall_mandsverk

In [12]:
with Session(engine) as session:
    # Join with select()
    stmt = (
        select(UnderstøtteTildelingsbrev, Vurdering)
        .join(Vurdering, UnderstøtteTildelingsbrev.prosjekt_id == Vurdering.prosjekt_id)
    )

    results = session.exec(stmt).all()
    for tildeling, vurdering in results:
        print(f"{tildeling.mål_1} (id {tildeling.mål_2}) has vurdering {vurdering.risiko_vurdering}")
    # for user, order in results:
    #     print(f"{user.name} (age {user.age}) bought {order.product} for {order.amount}")

2025-09-23 11:00:14,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-23 11:00:14,880 INFO sqlalchemy.engine.Engine SELECT portfolio.[UnderstøtteTildelingsbrev].strategisk_forankring_id, portfolio.[UnderstøtteTildelingsbrev].[mål_1_beskrivelse], portfolio.[UnderstøtteTildelingsbrev].[mål_2_beskrivelse], portfolio.[UnderstøtteTildelingsbrev].[mål_3_beskrivelse], portfolio.[UnderstøtteTildelingsbrev].[mål_1], portfolio.[UnderstøtteTildelingsbrev].[mål_2], portfolio.[UnderstøtteTildelingsbrev].[mål_3], portfolio.[UnderstøtteTildelingsbrev].sist_endret, portfolio.[UnderstøtteTildelingsbrev].er_gjeldende, portfolio.[UnderstøtteTildelingsbrev].prosjekt_id, portfolio.[Vurdering].vurdering_id, portfolio.[Vurdering].gruppe, portfolio.[Vurdering].pulje, portfolio.[Vurdering].risiko_vurdering, portfolio.[Vurdering].sist_endret AS sist_endret_1, portfolio.[Vurdering].er_gjeldende AS er_gjeldende_1, portfolio.[Vurdering].mscw, portfolio.[Vurdering].prosjekt_id AS prosjekt_id_1 
FROM portfo

In [4]:
from sqlalchemy.orm import selectinload

with Session(engine) as session:
    stmt = select(Client).options(selectinload(Client.orders))
    users = session.exec(stmt).all()

    for u in users:
        for order in u.orders:
            print(f"{u.name} -> {order.product} (${order.amount})")

AttributeError: orders

In [33]:
from sqlmodel import SQLModel, Field, Relationship
class Client(SQLModel, table=True):
    __tablename__ = "new_table_id"
    __table_args__ = {"schema": "dbo"}

    id: uuid.UUID = Field(
        default_factory=uuid.uuid4,
        sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
    )
    name: str
    age: int

    # one-to-many relationship
    orders: list["Orders"] = Relationship(back_populates="user")


class Orders(SQLModel, table=True):
    __tablename__ = "orders_test"
    __table_args__ = {"schema": "dbo"}

    id: uuid.UUID = Field(
        default_factory=uuid.uuid4,
        sa_column=Column(UNIQUEIDENTIFIER, primary_key=True, default=uuid.uuid4),
    )
    amount: int
    product: str
    

    customer_id: uuid.UUID = Field(
        foreign_key="dbo.users.id",  # 👈 link to users
    )

    user: User = Relationship(back_populates="orders")

InvalidRequestError: Table 'dbo.new_table_id' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [74]:
import urllib
driver_name = '{ODBC Driver 18 for SQL Server}'
server_name = os.getenv("SERVER")
database_name = os.getenv("DATABASE")
connection_string = 'Driver={};Server=tcp:{}.database.windows.net,1433;Database={};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'.format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
url = "mssql+pyodbc:///?odbc_connect={0}".format(params)
engine = create_engine(url)

In [29]:
uuid.uuid4()

UUID('8fc8caae-bf98-4e9f-9f2f-ce2c028d6856')

In [77]:
url

'mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC%20Driver%2018%20for%20SQL%20Server%7D%3BServer%3Dtcp%3Ab5li4afpbdwezicwwnkeoubzse-m3nesqac3ueepkmopydvg35o3y.datawarehouse.fabric.microsoft.com.database.windows.net%2C1433%3BDatabase%3DPortfolioWarehouse%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dno%3BConnection%20Timeout%3D30'

In [ ]:
import pyodbc
from sqlalchemy import create_engine
from azure.identity import DefaultAzureCredential

# Get an access token using DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token("https://database.windows.net/.default").token

# Construct the connection string using the access token
# Note: You might need to adjust the driver and server details based on your setup.
# The 'Authentication=ActiveDirectoryAccessToken' is crucial for AD token usage.
connection_string = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Authentication=ActiveDirectoryAccessToken;"
)

# Create the SQLAlchemy engine, passing the token in connect_args
# The token needs to be passed as a byte string for pyodbc.
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={pyodbc.quote_plus(connection_string)}",
    connect_args={"attrs_before": {pyodbc.SQL_COPT_SS_ACCESS_TOKEN: token.encode('utf-16-le')}}
)

# You can now use the engine to interact with your Azure SQL Database
# For example, to create a session:
# from sqlalchemy.orm import sessionmaker
# Session = sessionmaker(bind=engine)
# session = Session()

In [ ]:
pyodbc

In [56]:
engine = create_engine(odbc_str, echo=True, connect_args = {"attrs_before":{1256: token_struct}})

In [58]:
engine.connect()

DBAPIError: (pyodbc.Error) ('FA005', '[FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0) (SQLDriverConnect); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [25]:
result["access_token"].encode("UTF-16-LE")

b'e\x00y\x00J\x000\x00e\x00X\x00A\x00i\x00O\x00i\x00J\x00K\x00V\x001\x00Q\x00i\x00L\x00C\x00J\x00h\x00b\x00G\x00c\x00i\x00O\x00i\x00J\x00S\x00U\x00z\x00I\x001\x00N\x00i\x00I\x00s\x00I\x00n\x00g\x001\x00d\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00I\x00s\x00I\x00m\x00t\x00p\x00Z\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00J\x009\x00.\x00e\x00y\x00J\x00h\x00d\x00W\x00Q\x00i\x00O\x00i\x00J\x00o\x00d\x00H\x00R\x00w\x00c\x00z\x00o\x00v\x00L\x002\x00F\x00w\x00a\x00S\x005\x00m\x00Y\x00W\x00J\x00y\x00a\x00W\x00M\x00u\x00b\x00W\x00l\x00j\x00c\x00m\x009\x00z\x00b\x002\x00Z\x000\x00L\x00m\x00N\x00v\x00b\x00S\x00I\x00s\x00I\x00m\x00l\x00z\x00c\x00y\x00I\x006\x00I\x00m\x00h\x

In [22]:
credential.get_token("https://database.windows.net/.default").token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0IiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlhdCI6MTc1ODI3Njg3MywibmJmIjoxNzU4Mjc2ODczLCJleHAiOjE3NTgyODA3NzMsImFpbyI6ImsyUmdZRkJ4dTV0cDhrK2RJMW1SWGVQQ1luTTFBQT09IiwiYXBwaWQiOiI4NGRiYzc3Yy02MTMxLTQ4MmMtYTk5OC1mMzhjNGY4ZDgwZDQiLCJhcHBpZGFjciI6IjEiLCJncm91cHMiOlsiOTBjZTU1ZTAtYjliYi00ZGU5LTk5NDItZDNjYTY2YTViMDU1Il0sImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6IjI4YThlNDBlLTc5NmUtNGY4NC1iZmE0LTJlMDcxZjE1MmEzMSIsInJoIjoiMS5BVGtBRDFhT0FLOEk3RXlnVnJOVVIxQTVrZE1IS1FJYkRfZEl1dHdicHF1cmJXYXFBUUE1QUEuIiwic3ViIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwidGlkIjoiMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxIiwidXRpIjoieHczS0xwcHpZMGFvYmtNR1BXRjFBQSIsInZlciI6IjEuMCIsInhtc19mdGQiOi

In [18]:
credential.get_token("https://api.fabric.microsoft.com/.default").token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpYXQiOjE3NTgyNzY1NDEsIm5iZiI6MTc1ODI3NjU0MSwiZXhwIjoxNzU4MjgwNDQxLCJhaW8iOiJrMlJnWUpEczFwNXgzek9JUjczRmJhdkJBM1lIQUE9PSIsImFwcGlkIjoiODRkYmM3N2MtNjEzMS00ODJjLWE5OTgtZjM4YzRmOGQ4MGQ0IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwicmgiOiIxLkFUa0FEMWFPQUs4STdFeWdWck5VUjFBNWtRa0FBQUFBQUFBQXdBQUFBQUFBQUFDcUFRQTVBQS4iLCJzdWIiOiIyOGE4ZTQwZS03OTZlLTRmODQtYmZhNC0yZTA3MWYxNTJhMzEiLCJ0aWQiOiIwMDhlNTYwZi0wOGFmLTRjZWMtYTA1Ni1iMzU0NDc1MDM5OTEiLCJ1dGkiOiJabWZVMnZJeWZrQ2JtME1zMjl0d0FBIiwidmVyIjoiMS4wIiwieG1zX2Z0ZCI6Ik9RMW1KdGZEUEJ1R0NWVUxBcmF3TXh1TFh5T1BKS0lZVGYzLXhUcDBCeWNCYz

In [ ]:
("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')")

In [3]:
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import FileSystemClient

cred = ClientSecretCredential(tenant_id=azure_tenant_id,
                              client_id=azure_client_id,
                              client_secret=azure_client_secret)

file_system_client = FileSystemClient(
    account_url="https://onelake.dfs.fabric.microsoft.com",
    file_system_name="Project-portfolio",
    credential=cred)
paths = file_system_client.get_paths(path="/ProjectPortfolio.Lakehouse/Tables/")
for p in paths:
    print(p.name)

/ProjectPortfolio.Lakehouse/Tables/List_portfolio
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/36456da7f1414fef9e91d453e86ed50e.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/37bcc1a3c0c44d5db0bd051f51b33b57.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/5da9babcead1438da4c5ac088603611e.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000000.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000001.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000002.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary/_mashup_temporary_9fd6ce13-1c95-4abe-b7a9-3ff5a0ba37c7.version
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary/_mashup_temporary_b47d4639-d8f5-4fee-8ff8-c91251291059.

In [7]:
SQL_COPT_SS_ACCESS_TOKEN = 1256
def inject_azure_credential(credential, engine, token_url='https://database.windows.net/'):
    @event.listens_for(engine, 'do_connect')
    def do_connect(dialect, conn_rec, cargs, cparams):
        token = credential.get_token(token_url).token.encode('utf-16-le')
        token_struct = struct.pack(f'=I{len(token)}s', len(token), token)
        attrs_before = cparams.setdefault('attrs_before', {})
        attrs_before[SQL_COPT_SS_ACCESS_TOKEN] = bytes(token_struct)
        return dialect.connect(*cargs, **cparams)

In [5]:
cred.get_token("https://database.windows.net/.default").token.encode('utf-16-le')

b'e\x00y\x00J\x000\x00e\x00X\x00A\x00i\x00O\x00i\x00J\x00K\x00V\x001\x00Q\x00i\x00L\x00C\x00J\x00h\x00b\x00G\x00c\x00i\x00O\x00i\x00J\x00S\x00U\x00z\x00I\x001\x00N\x00i\x00I\x00s\x00I\x00n\x00g\x001\x00d\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00I\x00s\x00I\x00m\x00t\x00p\x00Z\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00J\x009\x00.\x00e\x00y\x00J\x00h\x00d\x00W\x00Q\x00i\x00O\x00i\x00J\x00o\x00d\x00H\x00R\x00w\x00c\x00z\x00o\x00v\x00L\x002\x00R\x00h\x00d\x00G\x00F\x00i\x00Y\x00X\x00N\x00l\x00L\x00n\x00d\x00p\x00b\x00m\x00R\x00v\x00d\x003\x00M\x00u\x00b\x00m\x00V\x000\x00I\x00i\x00w\x00i\x00a\x00X\x00N\x00z\x00I\x00j\x00o\x00i\x00a\x00H\x00R\x000\x00c\x00H\x00M\x

In [9]:
inject_azure_credential(cred, create_engine(connection_string))

NameError: name 'event' is not defined

In [8]:

engine2 = create_engine(connection_string, connect_args={'timeout': 120})

In [13]:
with engine2.connect() as connection:
    result = connection.execute(text("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')"))
    print(result)

In [12]:
with engine2.connect() as connection:
    result = connection.execute(text("SELECT * from [dbo].[new_table]"))
    print(result.all())

[(36, 'Ignacio')]


In [16]:
metadata = MetaData()

# Define the table structure
user_table = Table(
    '[dbo].[new_table]', metadata,
    Column('age', Integer),
    Column('name', String)
)

# Create the table in the database (if it doesn't exist)
metadata.create_all(engine2)

# Construct the INSERT statement
stmt = insert(user_table).values(age=26, name='alice@example.com')

# Execute the statement
with engine2.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
    # print(f"Inserted row with ID: {result.inserted_primary_key[0]}")

# # Inserting multiple rows
# data_to_insert = [
#     {'name': 'Bob', 'email': 'bob@example.com'},
#     {'name': 'Charlie', 'email': 'charlie@example.com'}
# ]
# stmt_multiple = insert(user_table).values(data_to_insert)

# with engine.connect() as conn:
#     conn.execute(stmt_multiple)
#     conn.commit()
#     print("Inserted multiple rows.")    

In [ ]:
result.

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [8]:
from msal import ConfidentialClientApplication
import os
TENANT_ID =  os.getenv("AZURE_TENANT_ID")
CLIENT_ID =  os.getenv("AZURE_CLIENT_ID")
CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET")

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://database.windows.net/.default"]
SCOPE_FABRIC = ["https://api.fabric.microsoft.com/.default"]


app = ConfidentialClientApplication(
    CLIENT_ID,
    authority=AUTHORITY,
    client_credential=CLIENT_SECRET,
)

result = app.acquire_token_for_client(scopes=SCOPE_FABRIC)


In [ ]:
app.get_

In [ ]:
token = credential.get_token(token_url).token.encode('utf-16-le')

In [7]:
result["access_token"]

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpYXQiOjE3NTgxOTI5NTUsIm5iZiI6MTc1ODE5Mjk1NSwiZXhwIjoxNzU4MTk2ODU1LCJhaW8iOiJrMlJnWU5oeHVraFFLdnY5TEsyMlYzNFRQdTdkQ1FBPSIsImFwcGlkIjoiODRkYmM3N2MtNjEzMS00ODJjLWE5OTgtZjM4YzRmOGQ4MGQ0IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwicmgiOiIxLkFUa0FEMWFPQUs4STdFeWdWck5VUjFBNWtRa0FBQUFBQUFBQXdBQUFBQUFBQUFDcUFRQTVBQS4iLCJzdWIiOiIyOGE4ZTQwZS03OTZlLTRmODQtYmZhNC0yZTA3MWYxNTJhMzEiLCJ0aWQiOiIwMDhlNTYwZi0wOGFmLTRjZWMtYTA1Ni1iMzU0NDc1MDM5OTEiLCJ1dGkiOiI3NHBpWXZXbXAwNnlQbDNGbmIwX0FBIiwidmVyIjoiMS4wIiwieG1zX2Z0ZCI6ImdFd0xIRmFWVWVORVJCZTM5VkowQ1lYZ24zTGJheTJPNEwyRGw1Q2VORUlCWm

In [ ]:
import requests
import msal

# 1. Acquire token
app = msal.ConfidentialClientApplication(
   client_id="YOUR_CLIENT_ID",
   client_credential="YOUR_CLIENT_SECRET",
   authority="https://login.microsoftonline.com/YOUR_TENANT_ID"
)
result = app.acquire_token_for_client(scopes=["https://api.fabric.microsoft.com/.default"])
token = result["access_token"]

# 2. Call API
headers = {"Authorization": f"Bearer {token}"}
resp = requests.get("https://api.fabric.microsoft.com/v1/connections", headers=headers)
resp.raise_for_status()